### Test

In [9]:
import torch
from PIL import Image
import requests
from transformers import Blip2Processor, Blip2Model

processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2Model.from_pretrained("Salesforce/blip2-opt-2.7b")
prompt = "Question: how many cats are there? Answer:"

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
inputs = processor(images=image, text=prompt, return_tensors="pt")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
qformer_outputs = model(**inputs)

In [19]:
for key in qformer_outputs: print(key)

logits
vision_outputs
qformer_outputs
language_model_outputs


In [ ]:
for key in qformer_outputs['qformer_outputs']['last_hidden_state']: print(key.shape)

In [24]:
qformer_outputs['qformer_outputs']['pooler_output'].shape

torch.Size([1, 768])

In [23]:
qformer_outputs['vision_outputs']['pooler_output']

tensor([[ 0.2400,  0.4323,  1.2008,  ..., -1.0685, -0.5133,  0.4215]],
       grad_fn=<NativeLayerNormBackward0>)

In [48]:
from PIL import Image
import requests
from transformers import AutoProcessor, LlavaForConditionalGeneration

model = LlavaForConditionalGeneration.from_pretrained("llava-hf/llava-1.5-7b-hf")
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")

prompt = "<image>\nUSER: What's the content of the image?\nASSISTANT:"
url = "https://www.ilankelman.org/stopsigns/australia.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(text=[prompt, prompt], images=[image, image], return_tensors="pt", truncation=True, max_length=150)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [49]:
inputs['input_ids'].shape

torch.Size([2, 20])

In [45]:
# Generate
generate_ids = model(**inputs, output_hidden_states=True)

In [41]:
generate_ids['hidden_states'][-1].mean(dim=1).shape

torch.Size([2, 4096])

In [31]:
generate_ids[1].shape

torch.Size([2, 595, 32064])

In [32]:
for key in generate_ids: print(key)

loss
logits
past_key_values
hidden_states


In [35]:
for key in generate_ids['hidden_states']: print(key.mean(dim=1).shape)

torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])


In [ ]:
import pandas as pd

In [18]:
#pd.read_csv('Embeddings_vlm/daquar/embeddings_blip2.csv').drop_duplicates().to_csv('Embeddings_vlm/daquar/embeddings_blip2.csv', index=False)

In [19]:
pd.read_csv('Embeddings_vlm/daquar/embeddings_clip.csv')

,question,image_id,answer,split,image_embedding_0,image_embedding_1,image_embedding_2,image_embedding_3,image_embedding_4,image_embedding_5,...,text_embedding_502,text_embedding_503,text_embedding_504,text_embedding_505,text_embedding_506,text_embedding_507,text_embedding_508,text_embedding_509,text_embedding_510,text_embedding_511
0,what is on the right side of the black telepho...,datasets/daquar/images/image3.png,desk,train,-0.034118,0.009930,-0.024554,0.060827,-0.007110,0.000265,...,0.009015,-0.010106,0.043033,-0.003312,-0.027812,0.009433,0.032928,0.020334,-0.013321,0.029527
1,what is in front of the white door on the left...,datasets/daquar/images/image3.png,telephone,train,-0.034118,0.009930,-0.024554,0.060827,-0.007110,0.000265,...,0.004590,0.004839,0.043577,0.056432,-0.047448,0.016236,0.015645,-0.048516,0.026294,-0.002189
2,what is on the desk in the image3 ?,datasets/daquar/images/image3.png,"book, scissor, papers, tape_dispenser",train,-0.034118,0.009930,-0.024554,0.060827,-0.007110,0.000265,...,0.013225,0.001205,0.034056,0.045491,-0.041755,0.030319,0.002640,-0.023645,0.010350,-0.001730
3,what is the largest brown objects in this imag...,datasets/daquar/images/image3.png,carton,train,-0.034118,0.009930,-0.024554,0.060827,-0.007110,0.000265,...,0.035328,0.009456,0.036832,0.042338,-0.039669,0.027131,-0.021927,-0.060768,-0.014135,-0.002478
4,what color is the chair in front of the white ...,datasets/daquar/images/image3.png,red,train,-0.034118,0.009930,-0.024554,0.060827,-0.007110,0.000265,...,-0.020662,-0.033852,0.057963,0.008730,-0.051158,0.016421,0.011833,-0.042573,0.004175,0.012011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12463,what is found below the chandelier in the imag...,datasets/daquar/images/image1448.png,table,test,-0.016653,0.029965,-0.019252,0.010936,0.023913,0.007702,...,-0.066609,-0.029039,0.028145,0.040204,-0.013461,0.046380,-0.001329,-0.008617,-0.011091,-0.005946
12464,what is on the floor in the image1449 ?,datasets/daquar/images/image1449.png,rug,test,-0.031555,0.034335,-0.015731,0.014051,0.013592,0.014325,...,-0.020197,-0.022020,0.026748,0.062430,-0.019698,0.046771,0.003754,-0.037050,-0.020266,0.005352
12465,what are around dining table in the image1449 ?,datasets/daquar/images/image1449.png,chair,test,-0.031555,0.034335,-0.015731,0.014051,0.013592,0.014325,...,-0.008075,-0.011334,0.033250,0.009502,0.011321,0.046418,-0.002084,0.025232,-0.022249,-0.026732
12466,what is at the opposite side of the dining tab...,datasets/daquar/images/image1449.png,decoration_item,test,-0.031555,0.034335,-0.015731,0.014051,0.013592,0.014325,...,-0.021455,-0.016994,0.041093,0.023230,-0.005219,0.029596,-0.025144,0.003514,-0.022612,0.001687


In [20]:
pd.read_csv('Embeddings_vlm/daquar/embeddings_clip.csv')#.drop_duplicates()

,question,image_id,answer,split,image_embedding_0,image_embedding_1,image_embedding_2,image_embedding_3,image_embedding_4,image_embedding_5,...,text_embedding_502,text_embedding_503,text_embedding_504,text_embedding_505,text_embedding_506,text_embedding_507,text_embedding_508,text_embedding_509,text_embedding_510,text_embedding_511
0,what is on the right side of the black telepho...,datasets/daquar/images/image3.png,desk,train,-0.034118,0.009930,-0.024554,0.060827,-0.007110,0.000265,...,0.009015,-0.010106,0.043033,-0.003312,-0.027812,0.009433,0.032928,0.020334,-0.013321,0.029527
1,what is in front of the white door on the left...,datasets/daquar/images/image3.png,telephone,train,-0.034118,0.009930,-0.024554,0.060827,-0.007110,0.000265,...,0.004590,0.004839,0.043577,0.056432,-0.047448,0.016236,0.015645,-0.048516,0.026294,-0.002189
2,what is on the desk in the image3 ?,datasets/daquar/images/image3.png,"book, scissor, papers, tape_dispenser",train,-0.034118,0.009930,-0.024554,0.060827,-0.007110,0.000265,...,0.013225,0.001205,0.034056,0.045491,-0.041755,0.030319,0.002640,-0.023645,0.010350,-0.001730
3,what is the largest brown objects in this imag...,datasets/daquar/images/image3.png,carton,train,-0.034118,0.009930,-0.024554,0.060827,-0.007110,0.000265,...,0.035328,0.009456,0.036832,0.042338,-0.039669,0.027131,-0.021927,-0.060768,-0.014135,-0.002478
4,what color is the chair in front of the white ...,datasets/daquar/images/image3.png,red,train,-0.034118,0.009930,-0.024554,0.060827,-0.007110,0.000265,...,-0.020662,-0.033852,0.057963,0.008730,-0.051158,0.016421,0.011833,-0.042573,0.004175,0.012011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12463,what is found below the chandelier in the imag...,datasets/daquar/images/image1448.png,table,test,-0.016653,0.029965,-0.019252,0.010936,0.023913,0.007702,...,-0.066609,-0.029039,0.028145,0.040204,-0.013461,0.046380,-0.001329,-0.008617,-0.011091,-0.005946
12464,what is on the floor in the image1449 ?,datasets/daquar/images/image1449.png,rug,test,-0.031555,0.034335,-0.015731,0.014051,0.013592,0.014325,...,-0.020197,-0.022020,0.026748,0.062430,-0.019698,0.046771,0.003754,-0.037050,-0.020266,0.005352
12465,what are around dining table in the image1449 ?,datasets/daquar/images/image1449.png,chair,test,-0.031555,0.034335,-0.015731,0.014051,0.013592,0.014325,...,-0.008075,-0.011334,0.033250,0.009502,0.011321,0.046418,-0.002084,0.025232,-0.022249,-0.026732
12466,what is at the opposite side of the dining tab...,datasets/daquar/images/image1449.png,decoration_item,test,-0.031555,0.034335,-0.015731,0.014051,0.013592,0.014325,...,-0.021455,-0.016994,0.041093,0.023230,-0.005219,0.029596,-0.025144,0.003514,-0.022612,0.001687
